In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import pickle
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 2816

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
file = open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/MultiModal/Early Fusion/Data/text_embed_pca_final.pickle', 'rb')
text_data = pickle.load(file)
file.close()

In [7]:
file = open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/MultiModal/Early Fusion/Data/visual_embed_padded_final.pickle', 'rb')
video_data = pickle.load(file)
file.close()

In [8]:
tv_data = {}
for scene in scenes:
    tv_data[scene] = np.concatenate((text_data[scene], video_data[scene]), axis=1)


In [9]:
tv_data['1_10004'].shape

(18, 2816)

In [10]:
def get_model_data(tv_data):
    model_data = pd.DataFrame(columns=['tv_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'tv_feature': tv_data[row['SCENE']], 
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [11]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data
   

In [12]:
class FNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'averaged_tv_feature']
            a=np.empty((1,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'averaged_tv_feature']
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)

class FNNNetSID(nn.Module):
    def __init__(self):
        super(FNNNetSID, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*desired_features, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*desired_features)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
class FNNNetSD(nn.Module):
    def __init__(self):
        super(FNNNetSD, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*(desired_features+1), hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*(desired_features+1))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
def predict_fnn(fnn_model, dataloader):
    prediction_list = []
    actual_list = []
    for data, target in dataloader:
        outputs = fnn_model(data)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
        actual_list.append(target)
    return prediction_list, actual_list

In [13]:
warnings.filterwarnings("ignore")

model_data = get_model_data(tv_data)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,tv_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-5.775393, -0.55747294, -5.9735856, 0.882094...",0.0,NONE,SHELDON,25
1,"[[-9.726368, -4.6866755, -11.944455, -6.425826...",0.0,NONE,PENNY,15
2,"[[-14.0798025, 4.135473, -8.056498, -4.865928,...",0.0,NONE,RAJ,21
3,"[[-5.3839507, 2.0448134, -13.128815, -6.819113...",1.0,PRO,HOWARD,7
4,"[[-3.3831522, 5.3072224, -5.980673, -1.8846271...",0.0,NONE,SHELDON,25


In [14]:
model_data['tv_feature'].iloc[0].shape

(18, 2816)

In [15]:
train_data, test_data = get_train_test_split(model_data, ['tv_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
fnn_train = train_data.copy()
fnn_test = test_data.copy()
fnn_train.reset_index(drop=True, inplace = True)
fnn_test.reset_index(drop=True, inplace = True)

        
fnn_train['averaged_tv_feature'] = fnn_train.loc[:, 'tv_feature']
for index, row in fnn_train.iterrows():
    av = row['averaged_tv_feature']
    fnn_train.at[index, "averaged_tv_feature"] = np.array([np.mean(av, axis=0)])

fnn_test['averaged_tv_feature'] = fnn_test.loc[:, 'tv_feature']
for index, row in fnn_test.iterrows():
    av = row['averaged_tv_feature']
    fnn_test.at[index, "averaged_tv_feature"] = np.array([np.mean(av, axis=0)])

fnn_train["sarcasm"] = fnn_train["sarcasm"].astype('int').to_numpy()
fnn_test["sarcasm"] = fnn_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 2) (961,) Test:  ((241, 2), (241,))


### Speaker Dependent

In [16]:
fnn_train_tensor = FNNTensorDataset(fnn_train[['averaged_tv_feature', 'speaker_encode', 'sarcasm']], True)
fnn_test_tensor = FNNTensorDataset(fnn_test[['averaged_tv_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 100
valid_size = 0.2

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

In [17]:
model = FNNNetSD()
print(model)

FNNNetSD(
  (fc1): Linear(in_features=2817, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(101):
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()*inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        test_loader_test = torch.utils.data.DataLoader(fnn_test_tensor, batch_size=fnn_test_tensor.__len__())
        predictions, actuals = predict_fnn(model, test_loader_test)
        print(classification_report(actuals[0].tolist(), predictions[0].tolist(), digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5939    0.8167    0.6877       120
           1     0.7105    0.4463    0.5482       121

    accuracy                         0.6307       241
   macro avg     0.6522    0.6315    0.6180       241
weighted avg     0.6525    0.6307    0.6177       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.6053    0.5750    0.5897       120
           1     0.5984    0.6281    0.6129       121

    accuracy                         0.6017       241
   macro avg     0.6018    0.6015    0.6013       241
weighted avg     0.6018    0.6017    0.6014       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.5882    0.5833    0.5858       120
           1     0.5902    0.5950    0.5926       121

    accuracy                         0.5892       241
   macro avg     0.5892    0.5892    0.5892       241
weighted avg     0.5892    0.5892    0.5892  